In [76]:
from pathlib import Path
import pandas as pd
import geopandas as gpd


In [77]:
csv_url =    'https://stg-arcgisazurecdataprod1.az.arcgis.com/exportfiles-1559-19445/Westminster_Parliamentary_Constituencies_Dec_2022_UK_BGC_-4629094885293682949.csv?sv=2018-03-28&sr=b&sig=CSlhL5mC0AaukzNQsUu1qRjfUnjbdcknlZmWGkdZV9g%3D&se=2024-02-08T01%3A40%3A26Z&sp=r'

In [78]:
data = pd.read_csv(csv_url, usecols=['PCON22CD', 'PCON22NM'], index_col=['PCON22CD'])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/giles/.local/share/virtualenvs/yff-data-ansOWYcy/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    A compiled code object, to be executed
  File "/tmp/ipykernel_22666/1900850390.py", line 1, in <module>
    data = pd.read_csv(csv_url, usecols=['PCON22CD', 'PCON22NM'], index_col=['PCON22CD'])
  File "/home/giles/.local/share/virtualenvs/yff-data-ansOWYcy/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 948, in read_csv
    stacklevel=find_stack_level(),
  File "/home/giles/.local/share/virtualenvs/yff-data-ansOWYcy/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 611, in _read
  File "/home/giles/.local/share/virtualenvs/yff-data-ansOWYcy/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 1448, in __init__
    ...
  File "/home/giles/.local/share/virtualenvs/yff-data-ansOWYcy/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 1705, in _mak

In [80]:
OUTPUT_DIR = Path('../../src/_data/areas/')
REF_DIR = OUTPUT_DIR.joinpath('reference')
DATA_DIR = OUTPUT_DIR.joinpath('data')

REF_DIR.mkdir(parents=True, exist_ok=True)

In [81]:
data.query('PCON22CD.str.startswith("E")').reset_index().to_json(REF_DIR / 'pcon.json', orient='records')

In [84]:
gejson_url = 'https://stg-arcgisazurecdataprod1.az.arcgis.com/exportfiles-1559-19445/Westminster_Parliamentary_Constituencies_Dec_2022_UK_BGC_7450510434469596417.geojson?sv=2018-03-28&sr=b&sig=pADvnyebzKGTdtdeTmHUmjiwZ2pFjShaQRw8SpjGO9o%3D&se=2024-02-08T02%3A43%3A50Z&sp=r'

In [110]:
geo = gpd.read_file(gejson_url, engine="pyogrio").to_crs('EPSG:4326').query('PCON22CD.str.startswith("E")').loc[:, ['PCON22CD', 'PCON22NM', 'geometry']]
geo.geometry = geo.geometry.simplify(tolerance=0.001, preserve_topology=True)
geo.crs = None

In [111]:
for idx, geo_code in geo.PCON22CD.items():
    d = DATA_DIR / geo_code
    d.mkdir(parents=True, exist_ok=True)
    geo.query(f'PCON22CD == @geo_code').to_file(d / 'map.geojson', engine="pyogrio", driver='GeoJSON')


/home/giles/.local/share/virtualenvs/yff-data-ansOWYcy/lib/python3.10/site-packages/pyogrio/raw.py:505: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
/home/giles/.local/share/virtualenvs/yff-data-ansOWYcy/lib/python3.10/site-packages/pyogrio/raw.py:505: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
/home/giles/.local/share/virtualenvs/yff-data-ansOWYcy/lib/python3.10/site-packages/pyogrio/raw.py:505: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
/home/giles/.local/share/virtualenvs/yff-data-ansOWYcy/lib/python3.10/site-packages/pyogrio/raw.py:505: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and m